In [7]:
from gibbs.dataclass import GibbsResult
import matplotlib.pyplot as plt
import numpy as np
from gibbs.learning.bayesian_learning import BayesianLearning
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory, DumbConstraintMatrixFactory
from qiskit.quantum_info import Statevector
from gibbs.learning.klocal_pauli_basis import KLocalPauliBasis
from scipy.linalg import block_diag
from scipy.sparse import bmat
from qiskit.quantum_info import state_fidelity
from gibbs.utils import number_of_elements, simple_purify_hamiltonian, spectral_dec,printarray, classical_learn_hamiltonian
import plotly_express as px
from pandas import DataFrame
# plt.rcParams['text.usetex'] = True

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
n=4
k_loc=2
prep_noise = 1e-1
basisH = KLocalPauliBasis(k_loc,n,False)
c_original = np.zeros(basisH.size)
c_original[basisH.pauli_to_num("XX"+"I"*(n-2)):basisH.pauli_to_num("I"*(n-2)+"XX")+1]=1
c_original[basisH.pauli_to_num("Z"+"I"*(n-1)):basisH.pauli_to_num("I"*(n-1)+"Z")+1]=-1/4
state = simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original),noise = 0)
px.bar(c_original)

In [31]:
def reconstruct(shots:float):    
    cmat_factory = ConstraintMatrixFactory(n,k_loc,k_loc+1)
    A = cmat_factory.create_cmat(state,shots=shots)
    s,v = spectral_dec(A)
    candidate = v[-1].copy(); candidate *= np.linalg.norm(c_original)
    if np.linalg.norm(c_original + candidate) < np.linalg.norm(c_original - candidate):
        candidate = -candidate
    return candidate, s

def error_estimate(sp_dec,shots):
    return np.sqrt(np.sum(1/sp_dec))*np.sqrt(1/shots)

In [33]:
candidate,sp_dec = reconstruct(shots:=1e3)
df ={"Candidate":np.real(candidate),"Original":np.real(classical_learn_hamiltonian(state,k_loc))}
px.bar(df,y=["Candidate","Original"],barmode="overlay").show()
print(f"The spectral gap is {sp_dec[-2]-sp_dec[-1]}, and the error estimate proportional to {error_estimate(sp_dec,shots)}.")
px.scatter(y=sp_dec,log_y=True).show()

The spectral gap is 0.02369228915861754, and the error estimate proportional to 0.30069643179770605.


In [35]:
df_noise = {
    "Shots":[],
    "Spectral Gap":[],
    "Fidelity":[],
    "Delta H":[],
    "Error Estimate":[]
}
for shots in np.logspace(4,6,num=100,base=10,dtype=int):
    print(f"Processing {shots}")
    can,sp_dec = reconstruct(shots=shots)
    df_noise["Shots"].append(shots)
    df_noise["Delta H"].append(np.linalg.norm(c_original-can))
    df_noise["Infidelity"].append(1-state_fidelity(state,simple_purify_hamiltonian(basisH.vector_to_pauli_op(can))))
    df_noise["Spectral Gap"].append(sp_dec[-2]-sp_dec[-1])
    df_noise["Error Estimate"].append(error_estimate(sp_dec,shots))

df_noise = DataFrame(df_noise)

Processing 10000
Processing 10476
Processing 10974
Processing 11497
Processing 12045
Processing 12618
Processing 13219
Processing 13848
Processing 14508
Processing 15199
Processing 15922
Processing 16681
Processing 17475
Processing 18307
Processing 19179
Processing 20092
Processing 21049
Processing 22051
Processing 23101
Processing 24201
Processing 25353
Processing 26560
Processing 27825
Processing 29150
Processing 30538
Processing 31992
Processing 33516
Processing 35111
Processing 36783
Processing 38535
Processing 40370
Processing 42292
Processing 44306
Processing 46415
Processing 48626
Processing 50941
Processing 53366
Processing 55908
Processing 58570
Processing 61359
Processing 64280
Processing 67341
Processing 70548
Processing 73907
Processing 77426
Processing 81113
Processing 84975
Processing 89021
Processing 93260
Processing 97700
Processing 102353
Processing 107226
Processing 112332
Processing 117681
Processing 123284
Processing 129154
Processing 135304
Processing 141747
Proces

In [42]:
px.scatter(df_noise,x="Shots",y=["Infidelity","Error Estimate"],log_x=True)